In this notebook I will try to process the drive data generated from my Automatic Link.
I will try to provide a general way for everyone to exclude some private location information so everybody may be able to share they data if they chose to.

In [20]:
# import what I need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [21]:
data = pd.read_csv("data/civic data.csv", parse_dates= [2,4])

Now the csv file contain location data of where and when I have been to. I will transform the location data, and for the most part I will just label my commute and group any other trip as 'other'. There are other location that I regularly go to, but I would rather not share here.

In [22]:
ls=[]
for i in range(15,23):
    ls.append(data.columns.values[i])
ls

['Start Location Lat',
 'Start Location Lon',
 'Start Location Accuracy (meters)',
 'End Location Lat',
 'End Location Lon',
 'End Location Accuracy (meters)',
 'Path',
 'Tags']

In [23]:
data=data.drop(ls, axis=1)

I also dropped the above columns. Obviously I don't want to show my location details; I will reuse the Tags column later on.


First put the your home street name. I found that because of  the phone GPS accuaracy, your location may be off. So if your home is on the concer of Washington Street and Main Street, it is better to put both of them there.
Based on how popular your street name is (Washington and Main is quite popular), you may need to include your street number. For me, my street name of interest is quite unique and I didn't have the problem.
You may use data['Start Location Name'].unique() to see if you will run into the problem of incorrectly labeling
The same idea applies to my work

In [24]:
home_name_1 ="Ironwood"
home_name_2 ="Mesquite"
work_name_1 = "Mountain Road"
work_name_2 = "East Pecos Road, Mesa, AZ 85212, USA"


Now we set the start and end location to just label them just as Home or Work

In [25]:
data.loc[(data['Start Location Name'].str.contains(work_name_1)==True)|(data['Start Location Name'].str.contains(work_name_2)==True)
         ,'Start Location Name'] ='Work'
data.loc[(data['Start Location Name'].str.contains(home_name_1)==True)|(data['Start Location Name'].str.contains(home_name_2)==True)
         ,'Start Location Name'] ='Home'
data.loc[(data['End Location Name'].str.contains(work_name_1)==True)|(data['End Location Name'].str.contains(work_name_2)==True)
         ,'End Location Name'] ='Work'
data.loc[(data['End Location Name'].str.contains(home_name_1)==True)|(data['End Location Name'].str.contains(home_name_2)==True)
         ,'End Location Name'] ='Home'

After we label them, we will make use of the Tags column to identify commute trips, and all other trips are under 'Other'

In [26]:
commute_mask = (data['Start Location Name'] =='Home') & (data['End Location Name'] =='Work')|(data['End Location Name'] =='Home') & (data['Start Location Name'] =='Work')
data.loc[commute_mask, 'Tags'] = 'commute'
data.Tags = data.Tags.fillna('Other')

In [27]:
data=data.drop(['Start Location Name','End Location Name','Vehicle'], axis=1)
data.to_csv('data/public_data.csv')